### MOVIE DATA ANALYTICS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file
#file_path = r"C:\Users\Owner\Desktop\Tyler\OneDrive\Excel Files\Movies Ranks.xlsm"  # Update this with your actual file path
file_path = r"C:\Users\jrtgr\OneDrive\Excel Files\Movies Ranks.xlsm"
xls = pd.ExcelFile(file_path)

# Load the "Movie Rankings" sheet
df_movies = pd.read_excel(xls, sheet_name="Movie Rankings")

# Data Cleaning: Remove unnamed columns
df_movies = df_movies.drop(columns=[col for col in df_movies.columns if "Unnamed" in col])

# Handling missing values
df_movies = df_movies.dropna(how='all')  # Remove completely empty rows

# Summary statistics
summary_stats = df_movies.describe()

# Save summary statistics as an Excel file
summary_file = "Summary_Statistics.xlsx"
summary_stats.to_excel(summary_file, index=True)

print(f"Summary statistics saved to {summary_file}")

# Top 10 Actors with Most Appearances
actor_columns = ['Actor', 'Actor 2', 'Actor 3', 'Actor 4']
all_actors = pd.concat([df_movies[col] for col in actor_columns])
top_actors = all_actors.value_counts().head(10)

# Distribution of Movie Scores
plt.figure(figsize=(8, 5))
sns.histplot(df_movies['Score'], bins=15, kde=True)
plt.title("Distribution of Movie Scores")
plt.xlabel("Score")
plt.ylabel("Count")
plt.grid(True)
plt.show()

# Trend: Average Movie Score by Year
df_movies['Year'] = pd.to_numeric(df_movies['Year'], errors='coerce')
yearly_avg_score = df_movies.groupby('Year')['Score'].mean()

plt.figure(figsize=(10, 5))
sns.lineplot(x=yearly_avg_score.index, y=yearly_avg_score.values, marker='o')
plt.title("Average Movie Score Over Time")
plt.xlabel("Year")
plt.ylabel("Average Score")
plt.grid(True)
plt.show()

# Top 10 Actors with Most Appearances
plt.figure(figsize=(10, 5))
sns.barplot(x=top_actors.values, y=top_actors.index, hue=top_actors.index, palette="viridis", legend=False)
plt.title("Top 10 Most Frequently Appearing Actors")
plt.xlabel("Number of Appearances")
plt.ylabel("Actor")
plt.grid(axis="x")
plt.show()

# 3️⃣ Movie Scores Over Time: Rolling Average (Smoother Trend Line)
df_movies_sorted = df_movies.sort_values("Year")
df_movies_sorted["Rolling_Avg"] = df_movies_sorted["Score"].rolling(window=5, min_periods=1).mean()

plt.figure(figsize=(10, 6))
sns.lineplot(x=df_movies_sorted["Year"], y=df_movies_sorted["Rolling_Avg"], marker='o', label="5-Year Rolling Avg", color="blue")
plt.title("Movie Scores Over Time (Rolling Average)")
plt.xlabel("Year")
plt.ylabel("Rolling Avg Score")
plt.grid(True)
plt.legend()
plt.show()


### MOVIE RATING PREDICTOR

In [3]:
import pandas as pd
import numpy as np
import os
import csv
from tkinter import Tk, Label, Entry, Button, StringVar, messagebox, ttk, Listbox

from collections import defaultdict, Counter

# **Load Movie Data**
file_path = r"C:\Users\jrtgr\OneDrive\Projects\Movies\Movies Ranks.xlsm"
#file_path = r"C:\Users\jrtgr\OneDrive\Projects\Movies\Movies Ranks.xlsm"

xls = pd.ExcelFile(file_path)
df_movies = pd.read_excel(xls, sheet_name="Movie Rankings")

# **Extract Genre and Actor Columns**
genre_columns = ["Genre1", "Genre2", "Genre3"]
actor_columns = ["Actor", "Actor 2", "Actor 3", "Actor 4"]

df_movies[genre_columns] = df_movies[genre_columns].astype(str).replace("nan", None)
df_movies[actor_columns] = df_movies[actor_columns].astype(str).replace("nan", None)
df_movies["Genres"] = df_movies[genre_columns].apply(lambda x: ', '.join(x.dropna()), axis=1)

# **Compute Average Ratings for Genres and Actors**
genre_scores = defaultdict(list)
actor_scores = defaultdict(list)

for _, row in df_movies.iterrows():
    for genre in row[genre_columns]:
        if genre and genre != "None":
            genre_scores[genre].append(row["Score"])
    for actor in row[actor_columns]:
        if actor and actor != "None":
            actor_scores[actor].append(row["Score"])

# **Create Dynamic Weights**
genre_weights = {genre: sum(scores) / len(scores) for genre, scores in genre_scores.items()}
actor_avg_rating = {actor: sum(scores) / len(scores) for actor, scores in actor_scores.items()}

# **List of Unique Genres & Frequent Actors**
all_genres = sorted(genre_weights.keys())
actor_counts = Counter(df_movies['Actor'].dropna().tolist() + df_movies['Actor 2'].dropna().tolist())
frequent_actors = sorted([actor for actor, count in actor_counts.items() if count >= 2])

# **Boost Factors**
genre_boost_factor = 1.9
actor_boost_factor = 1.5
year_decay_factor = 0.01

# **Year Weight Adjustment**
def apply_year_boost(year):
    latest_year = max(df_movies["Year"].dropna())  
    return 1 - (latest_year - year) * year_decay_factor  

# **Encode Genre & Actor Influence**
def encode_genres_boosted(selected_genres):
    valid_genres = [genre for genre in selected_genres if genre in genre_weights]
    if not valid_genres:
        return np.mean(list(genre_weights.values()))  
    encoded = [genre_weights.get(genre, np.mean(list(genre_weights.values()))) * genre_boost_factor for genre in valid_genres]
    return np.mean(encoded)

def get_actor_rating_boosted(actor):
    if actor not in actor_avg_rating:
        return np.mean(list(actor_avg_rating.values()))
    base_score = actor_avg_rating[actor]
    return base_score * actor_boost_factor

# **Create GUI**
root = Tk()
root.title("Movie Score Predictor")
root.geometry("400x750")
root.configure(bg="#2C2F33")

# **Input Handling Functions**
def handle_focus_in(entry, default_value):
    if entry.get() == default_value:
        entry.delete(0, "end")

def handle_focus_out(entry, default_value):
    if entry.get() == "":
        entry.insert(0, default_value)

# **User Inputs**
Label(root, text="Enter IMDb Rating (0.0):", fg="white", bg="#2C2F33", font=("Arial", 10, "bold")).pack(pady=(5, 0))
imdb_entry = Entry(root, bg="#F0F0F0", font=("Arial", 10))
imdb_entry.insert(0, "0.0")
imdb_entry.bind("<FocusIn>", lambda event: handle_focus_in(imdb_entry, "0.0"))
imdb_entry.bind("<FocusOut>", lambda event: handle_focus_out(imdb_entry, "0.0"))
imdb_entry.pack()

Label(root, text="Enter Rotten Tomatoes Score (00):", fg="white", bg="#2C2F33", font=("Arial", 10, "bold")).pack(pady=(5, 0))
rt_entry = Entry(root, bg="#F0F0F0", font=("Arial", 10))
rt_entry.insert(0, "00")
rt_entry.bind("<FocusIn>", lambda event: handle_focus_in(rt_entry, "00"))
rt_entry.bind("<FocusOut>", lambda event: handle_focus_out(rt_entry, "00"))
rt_entry.pack()

Label(root, text="Enter Release Year (0000):", fg="white", bg="#2C2F33", font=("Arial", 10, "bold")).pack(pady=(5, 0))
year_entry = Entry(root, bg="#F0F0F0", font=("Arial", 10))
year_entry.insert(0, "0000")
year_entry.bind("<FocusIn>", lambda event: handle_focus_in(year_entry, "0000"))
year_entry.bind("<FocusOut>", lambda event: handle_focus_out(year_entry, "0000"))
year_entry.pack()

# **Genre Selection**
Label(root, text="Select Up to 3 Genres:", fg="white", bg="#2C2F33", font=("Arial", 10, "bold")).pack(pady=(5, 0))
genre_var1, genre_var2, genre_var3 = StringVar(value="None"), StringVar(value="None"), StringVar(value="None")
for var in [genre_var1, genre_var2, genre_var3]:
    ttk.Combobox(root, textvariable=var, values=["None"] + all_genres, state="readonly").pack(pady=2)

# **Searchable Actor Selection**
def search_actor(event, search_var, listbox):
    typed = search_var.get().lower()
    filtered_actors = [actor for actor in frequent_actors if typed in actor.lower()]
    listbox.delete(0, "end")
    for actor in filtered_actors:
        listbox.insert("end", actor)
    listbox.pack()

def select_actor(event, search_var, listbox):
    selected_actor = listbox.get(listbox.curselection())
    search_var.set(selected_actor)
    listbox.pack_forget()

# **Actor Selection**
actor_search_var1, actor_search_var2 = StringVar(), StringVar()
for i, actor_var in enumerate([actor_search_var1, actor_search_var2], 1):
    Label(root, text=f"Search for Lead Actor {i}:", fg="white", bg="#2C2F33", font=("Arial", 10, "bold")).pack(pady=(5, 0))
    actor_entry = Entry(root, textvariable=actor_var, bg="#F0F0F0", font=("Arial", 10))
    actor_entry.pack()
    actor_listbox = Listbox(root, height=5)
    actor_entry.bind("<KeyRelease>", lambda event, var=actor_var, lb=actor_listbox: search_actor(event, var, lb))
    actor_listbox.bind("<<ListboxSelect>>", lambda event, var=actor_var, lb=actor_listbox: select_actor(event, var, lb))

# **Movie Name Input**
Label(root, text="Enter Movie Name:", fg="white", bg="#2C2F33", font=("Arial", 10, "bold")).pack(pady=(5, 0))
movie_name_entry = Entry(root, bg="#F0F0F0", font=("Arial", 10))
movie_name_entry.pack()

# **File to Store Predictions**
prediction_file = "Predicted Movie Scores.csv"

def save_prediction(movie_name, predicted_score, actor1, actor2, imdb, rt, year, genres):
    file_exists = os.path.isfile(prediction_file)
    
    with open(prediction_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Movie Name", "Predicted Score", "Actor 1", "Actor 2", "IMDb Rating", "Rotten Tomatoes Score", "Year", "Genres"])
        writer.writerow([movie_name, predicted_score, actor1, actor2, imdb, rt, year, ", ".join(genres)])

def movie_exists(movie_name):
    if not os.path.isfile(prediction_file):
        return False
    with open(prediction_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  # Skip the header
        for row in reader:
            if row and row[0].strip().lower() == movie_name.lower():
                return True
    return False

def predict_score():
    imdb, rt, year = float(imdb_entry.get()), int(rt_entry.get()), int(year_entry.get())
    selected_genres = [g for g in [genre_var1.get(), genre_var2.get(), genre_var3.get()] if g != "None"]
    selected_actors = [actor_search_var1.get(), actor_search_var2.get()]
    
    predicted_score = round((imdb + rt / 10 + encode_genres_boosted(selected_genres) + np.mean([get_actor_rating_boosted(a) for a in selected_actors]) + apply_year_boost(year)) / 5, 2)

    movie_name = movie_name_entry.get().strip()

    if movie_name:
        if movie_exists(movie_name):
            messagebox.showinfo("Predicted Score", f"🎬 Expected Movie Score: {predicted_score}\n\n⚠️ Movie already exists in CSV, not saving.")
        else:
            save_prediction(movie_name, predicted_score, selected_actors[0], selected_actors[1], imdb, rt, year, selected_genres)
            messagebox.showinfo("Predicted Score", f"🎬 Expected Movie Score: {predicted_score}\n\n✅ Saved successfully!")
    else:
        messagebox.showwarning("No Movie Name", "Please enter a movie name to save the prediction.")
        
# **Predict & Save Button**
Button(root, text="Predict & Save Score", command=predict_score, bg="#7289DA", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

root.mainloop()

c:\Users\jrtgr\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Slicer List extension is not supported and will be removed
  for idx, row in parser.parse():


### MOVIES TO WATCH

In [ ]:
import pandas as pd

# Load the CSV file (update the file path if needed)
file_path = r"C:\Users\Owner\Desktop\Tyler\OneDrive\Movie Predictor\Predicted Movie Scores.csv"
df = pd.read_csv(file_path)
# Get the top 10 movies based on the Predicted Score
top_10_movies = df.nlargest(20, 'Predicted Score')

# Reset the index and add a ranking column
top_10_movies.reset_index(drop=True, inplace=True)
top_10_movies.index += 1  # Start ranking from 1

# Display the top 10 movies with rankings
print(top_10_movies.to_string(index=True))

### Mobile Version

In [ ]:
pip install streamlit pandas numpy openpyxl

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install --no-cache-dir pandas streamlit


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np

# 🎬 **Title**
st.title("🎬 Movie Score Predictor")

# **Default File Path**
DEFAULT_FILE_PATH = "/mnt/data/Movies Ranks.xlsm"

# **Function to Load Movie Data**
def load_movie_data(file_path):
    xls = pd.ExcelFile(file_path)
    return pd.read_excel(xls, sheet_name="Movie Rankings")

# **Load the default file initially**
df_movies = load_movie_data(DEFAULT_FILE_PATH)

# **Optional File Upload**
uploaded_file = st.file_uploader("Upload a new Movies Ranks Excel file (Optional)", type=["xlsm", "xlsx"])

if uploaded_file is not None:
    df_movies = load_movie_data(uploaded_file)
    st.success("✅ New file uploaded successfully!")

# **Compute Average Ratings for Genres and Actors**
genre_columns = ["Genre1", "Genre2", "Genre3"]
actor_columns = ["Actor", "Actor 2", "Actor 3", "Actor 4"]

genre_scores = {}
actor_scores = {}

for _, row in df_movies.iterrows():
    for genre in row[genre_columns]:
        if genre and genre != "None":
            genre_scores.setdefault(genre, []).append(row["Score"])
    for actor in row[actor_columns]:
        if actor and actor != "None":
            actor_scores.setdefault(actor, []).append(row["Score"])

genre_weights = {genre: np.mean(scores) for genre, scores in genre_scores.items()}
actor_avg_rating = {actor: np.mean(scores) for actor, scores in actor_scores.items()}

# **User Inputs**
imdb = st.number_input("Enter IMDb Rating", 0.0, 10.0, 5.0)
rt = st.number_input("Enter Rotten Tomatoes Score", 0, 100, 50)
year = st.number_input("Enter Release Year", 1900, 2025, 2022)

selected_genres = st.multiselect("Select Genres", list(genre_weights.keys()))
selected_actors = st.multiselect("Select Actors", list(actor_avg_rating.keys()))

# **Prediction Function**
if st.button("Predict Score"):
    predicted_score = (imdb + rt / 10 + np.mean([genre_weights.get(g, 5) for g in selected_genres]) + 
                      np.mean([actor_avg_rating.get(a, 5) for a in selected_actors])) / 4
    st.success(f"Predicted Movie Score: {round(predicted_score, 2)} 🎥")
